In [ ]:
import numpy as np
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import time

In [ ]:
# CUDA 内核函数：矩阵乘法计算
kernel_code = """
__global__ void matmul_optimized(float *A, float *B, float *C, int N) {
    int row = threadIdx.x + blockIdx.x * blockDim.x;
    int col = threadIdx.y + blockIdx.y * blockDim.y;

    if (row < N && col < N) {
        float value = 0.0f;
        for (int k = 0; k < N; k++) {
            value += A[row * N + k] * B[k * N + col];
        }
        C[row * N + col] = value;
    }
}
"""

In [ ]:
# 创建 CUDA 模块
mod = SourceModule(kernel_code)
matmul_optimized = mod.get_function("matmul_optimized")

In [ ]:
# 主机数据
N = 1024
A = np.random.rand(N, N).astype(np.float32)
B = np.random.rand(N, N).astype(np.float32)
C = np.zeros((N, N), dtype=np.float32)

In [ ]:
# 设备数据
A_gpu = cuda.to_device(A)
B_gpu = cuda.to_device(B)
C_gpu = cuda.to_device(C)

In [ ]:
# 定义线程快大小和网格大小
block_size = (32, 32, 1)   # 每个线程快包含 32 * 32 个线程
grid_size = (N // block_size[0], N // block_size[1], 1)

In [ ]:
start_time = time.time()    # 计算开始时间

In [ ]:
# 执行 CUDA 内核函数
matmul_optimized(A_gpu, B_gpu, C_gpu, np.int32(N), block=block_size, grid=grid_size)

In [ ]:
C_gpu.get(C)

In [ ]:
end_time = time.time()

In [ ]:
# 输出部分计算结果
print("Matrix multiplication took {:.4f} seconds.".format(end_time-start_time))
print("First 5 rows and columns of the result:")
print(C[:5, :5])

In [ ]:
# 计算 性能指标
# 浮点预算量
flop_count = 2 * N ** 3

In [ ]:
# 计算吞吐量（单位：GFLOP/S）
execution_time = end_time - start_time
gflops = flop_count / execution_time / 1e9
print(f"浮点运算性能：{gflops:.4f} GFLOP/s")

In [ ]:
# 计算内存带块（单位：GB/s)
# 读/写 A、B、C 矩阵，每个元素的读/写到需要消耗 4 字节的存储量
memory_bandwidth = 2 * N * N * N * 4 / execution_time / 1e9
print(f"内存带宽： {memory_bandwidth:.4f} GB/s")

识别性能瓶颈：

+ 内存访问瓶颈：特别是全局内存访问；

+ 计算瓶颈

    程序的计算任务没有有效被分配给GPU的计算资源，则可能会导致计算单元的空闲

+ 线程调度瓶颈

    如果线程调度设置得不合理，则可能会导致部分线程处于等待状态。 使用 CUDA 流和事件管理可以优化线程的执行顺序，避免线程间的竞争和阻塞

+ 同步瓶颈



计算性能与效率指标

+ 浮点运算性能

    浮点运算性能是指每秒完成的浮点运算次数，通常以每秒浮点运算数(FLOP/s)表示；

+ 内存带宽

    内存带宽是指GPU与内存之间的数据传输速率；

+ 吞吐量

    指的是单位时间内系统能够处理的计算任务量。吞吐量越高，GPU在执行任务时的效率越高。它与GPU的核心数、时钟频率、内存带宽等因素密切相关

+ GPU 利用率

    GPU利用率是衡量GPU计算资源使用情况的重要指标；

如何计算 GPU 效率

$GPU_E = \frac{实际完成工作量}{GPU理论完成最大工作量}$